# Data Prep

Primeiro, vamos importar a biblioteca pandas, talvez a biblioteca mais famosa do Python para análise de dados. Além disso, vamos dar um alias para ela de "pd", como já é comum na comunidade de cientistas de dados.

In [1]:
import pandas as pd

Agora, vamos importar a nossa base de dados, um arquivo 'csv', com os campos separados por vírgulas.

Uma maneira muito fácil de fazer isso é utilizar o método "read_csv" do pandas e passar o caminho do arquivo.

Caso utilize um sistema Windows, é importante um passar uma letra 'r' antes do caminho do arquivo, assim ele vai ler as '\' do seu caminho corretamente.

In [2]:
df = pd.read_csv(r'C:\Users\jcalv\Desktop\train.csv')

Utilizando o método .head(), podemos verificar as 5 primeiras linhas do nosso dataframe e verificar que tipo de informação está disponível.

In [3]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


O atributo .shape retorna o tamanho do dataframe, no caso desse exemplo são: 913.000 linhas x 4 colunas

In [4]:
df.shape

(913000, 4)

Outro atributo muito importante de verificar é os tipos das variáveis. Podemos verificar que o campo ***date***, foi detectado incorretamente como um ***object***, um formato texto.

In [5]:
df.dtypes

date     object
store     int64
item      int64
sales     int64
dtype: object

Para executarmos as operações nesse campo, vamos converter para o formato datetime. Para isso podemos utilizar a função ***to_datetime*** do pandas.

In [6]:
df['date'] = pd.to_datetime(df['date'])

Verificando os tipos novamente, podemos notar que o campo ***date*** foi corrigido.

In [7]:
df.dtypes

date     datetime64[ns]
store             int64
item              int64
sales             int64
dtype: object

# Feature Engineering

Um processo muito importante em um fluxo de Machine Learning é a criação de variáveis. Elas vão explicar o comportamento dos nossos dados gerando padrões para serem identificados pelos algoritmos de Machine Learning.

In [8]:
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year
df['dayofweek'] = df.date.dt.dayofweek
df['dayofyear'] = df.date.dt.dayofyear
df['weekofyear'] = df.date.dt.weekofyear

In [9]:
df.head()

,date,store,item,sales,day,month,year,dayofweek,dayofyear,weekofyear
0,2013-01-01,1,1,13,1,1,2013,1,1,1
1,2013-01-02,1,1,11,2,1,2013,2,2,1
2,2013-01-03,1,1,14,3,1,2013,3,3,1
3,2013-01-04,1,1,13,4,1,2013,4,4,1
4,2013-01-05,1,1,10,5,1,2013,5,5,1


In [10]:
df.drop('date',axis=1,inplace=True)

# Treino e Teste

Verificando as datas da nossa base notamos que as informações vão do primeiro dia de 2013 até o último dia de 2017.

In [11]:
df.year.unique()

array([2013, 2014, 2015, 2016, 2017], dtype=int64)

Podemos utilizar um método de treino e teste conhecido como ***Out Of Time***, essa estratégia consiste em deixar período no tempo para treino e outro período para teste. No caso da nossa base, temos 5 anos, se reservarmos o último ano para teste, teremos 80% dos anos treinando o algoritmo e o restante 20% são utilizados para testar o modelo.

In [12]:
train = df[df.year < 2017]
test = df[df.year == 2017]

Agora vamos seprar a nossa variável resposta ***sales***, em uma variável exclusiva e dar o nome de ***y_train*** e ***y_test***.

In [13]:
y_train = train['sales']
y_test = test['sales']

Para os dados de treinos, utilizaremos todas as variáveis, menos ***sale***, pois essa é a variável que estamos tentando prever.

In [14]:
X_train = train.drop('sales',axis=1)
X_test = test.drop('sales',axis=1)

In [15]:
X_train.head()

,store,item,day,month,year,dayofweek,dayofyear,weekofyear
0,1,1,1,1,2013,1,1,1
1,1,1,2,1,2013,2,2,1
2,1,1,3,1,2013,3,3,1
3,1,1,4,1,2013,4,4,1
4,1,1,5,1,2013,5,5,1


# Regressão Linear

Agora vamos tentar prever as vendas de uma data utilizando a implementação de Regressão Linear disponível na biblioteca Scikit Learn.

In [16]:
from sklearn.linear_model import LinearRegression

Primeramente vamos instanciar o algoritmo em uma variável.

In [17]:
rg = LinearRegression()

Agora, vamos treinar a variável com os nossos dados de treino.

In [18]:
rg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Com o nosso modelo já calculado na variável ***rg*** podemos prever os resultados para a nossa base teste.

In [19]:
predicted = rg.predict(X_test)

In [20]:
predicted

array([69.6467017 , 50.16724615, 53.26180544, ..., 70.11403413,
       73.20859342, 76.3031527 ])

Para comparar os resultados previstos com os realizados, podemos criar um dataframe e colocar cada informação lado a lado.

In [21]:
resultados = pd.DataFrame()
resultados['realizado'] = y_test
resultados['previsto'] = predicted
resultados['erro'] = predicted - y_test

In [22]:
resultados.head()

,realizado,previsto,erro
1461,19,69.646702,50.646702
1462,15,50.167246,35.167246
1463,10,53.261805,43.261805
1464,16,56.356365,40.356365
1465,14,59.450924,45.450924


Agora, como métrica de comparação podemos utilizar o RMSE(Root Mean Squared Error).

In [23]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [24]:
np.sqrt(mean_squared_error(y_test,predicted))

31.01544408334406

# Gradient Boosting

Para comparar a performance com um modelo não linear, pode utilizar o Gradient Boosting, também da biblioteca Scikit Learn.

In [25]:
from sklearn.ensemble import GradientBoostingRegressor

In [26]:
gb = GradientBoostingRegressor()

In [27]:
gb.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [28]:
predicted = gb.predict(X_test)

In [29]:
resultados = pd.DataFrame()
resultados['realizado'] = y_test
resultados['previsto'] = predicted
resultados['erro'] = predicted - y_test

In [30]:
resultados.head()

,realizado,previsto,erro
1461,19,17.686237,-1.313763
1462,15,8.216029,-6.783971
1463,10,12.806161,2.806161
1464,16,12.806161,-3.193839
1465,14,13.341892,-0.658108


Verificando o Root Mean Squared Erro, podemos notar que quase dobramos a performance.

In [31]:
np.sqrt(mean_squared_error(y_test,predicted))

18.734139942173794